## Load Data

In [ ]:
import pandas as pd
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Load the dataset
data = pd.read_csv('data/data.csv')

# Convert categorical features to numeric using label encoding
categorical_features = ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService','OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV','StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod']  # include more if needed
encoder = LabelEncoder()

for feature in categorical_features:
    data[feature] = encoder.fit_transform(data[feature])

# convert total charges to numeric
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

# handle missing values, if any
data = data.dropna()

# normalize numeric features to scale between 0 and 1
scaler = MinMaxScaler()
data[['tenure', 'MonthlyCharges', 'TotalCharges']] = scaler.fit_transform(
    data[['tenure', 'MonthlyCharges', 'TotalCharges']])

# Separate features and target
X = data.drop('Churn', axis=1)
X = X.drop("customerID",axis=1)
y = data['Churn']

# Convert yes/no to 1/0 for target variable
y = y.map({'Yes': 1, 'No': 0})

# Split data into train and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

# Convert to tensors
X_train = torch.from_numpy(X_train.values).float()
y_train = torch.from_numpy(y_train.values).long()

X_val = torch.from_numpy(X_val.values).float()
y_val = torch.from_numpy(y_val.values).long()


In [ ]:
X_train.size()

## LSTMChurnPredictor with 3-D shape

```python
from torch import nn


class LSTMChurnPredictor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMChurnPredictor, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))  # out shape : (batch_size, seq_length, hidden_size)
        out = out[:, -1, :]
        out = self.fc(out)

        return out
```

## LSTMChurnPredictor with 2-D shape for loop train

In [ ]:
class LSTMChurnPredictor(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMChurnPredictor, self).__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=False)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, 1, self.hidden_size).to(x.device)  # note the 1 here
        c0 = torch.zeros(self.num_layers, 1, self.hidden_size).to(x.device)  # and here

        out, _ = self.lstm(x.unsqueeze(1), (h0, c0))  # out shape : (seq_length, 1, hidden_size)
        out = out[:, -1, :]
        out = self.fc(out)

        return out


## Loop train

In [ ]:
# Import necessary PyTorch libraries
import torch
from torch import optim, nn
from torch.utils.data import TensorDataset, DataLoader

# Number of input features
input_size = 19
# Number of hidden units for the LSTM
hidden_size = 32
# Number of LSTM layers
num_layers = 2
# Binary classification output
output_size = 1
# Define model
model = LSTMChurnPredictor(input_size, hidden_size, num_layers, output_size)

# Running on a CPU; for running on GPU, use model.to('cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Choose the hyperparameters for training: 
num_epochs = 100
learning_rate = 0.001

# Define the loss function and the optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# DataLoader for your training and validation data
train_data = TensorDataset(X_train, y_train)
val_data = TensorDataset(X_val, y_val)
batch_size = 32
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size=batch_size)

# Training loop
for epoch in range(num_epochs):
    for i, (customer_data, labels) in enumerate(train_loader):
        customer_data = customer_data.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(customer_data)
        loss = criterion(outputs, labels.float().view(-1, 1))

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        for customer_data, labels in val_loader:
            outputs = model(customer_data)
            val_loss = criterion(outputs, labels.float().view(-1, 1))
    model.train()

    # Print loss for every epoch
    print('Epoch [{}/{}], Loss: {:.4f}, Val Loss: {:.4f}'.format(epoch + 1, num_epochs, loss.item(), val_loss.item()))


## Predict churn

In [ ]:
def predict(model, input_data):
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        predictions = model(input_data)
        probs = torch.sigmoid(predictions).numpy()
    return probs

input_data = X_train

# Get the probabilities with the helper function
probs = predict(model, input_data)


In [ ]:
probs

In [ ]:
predicted_classes = (probs > 0.5)

In [ ]:
predicted_classes

## Save the model

In [ ]:
PATH = "model/model.pth"

torch.save(model.state_dict(), PATH)

In [ ]:
df = pd.DataFrame(data=probs, columns=["Predicted Probability"])
df.to_csv('./model/predictions.csv', index=True)